In [4]:
from src.utils.hybridsearch import run_hybrid_search
res = run_hybrid_search(
    query="có ip 15 128 ko bro",
    k=20,
    is_bm25_enable=True
)
res

[Document(metadata={'id': '5b9657ddffcdc8b0c9f23f78de53d319', 'source': 'src/data/hoanghamobile_with_summary.csv', '_id': '666baeb69793e149fe739410'}, page_content="Tên sản phẩm: điện thoại iphone 15 pro (128gb) - chính hãng vn/a | Khuyến mãi: - KM 1<br>- Giảm thêm 100.000đ khi khách hàng thanh toán bằng hình thức chuyển khoản ngân hàng khi mua iPhone 15 Series.<br>- KM 2<br>- Ưu đãi trả góp 0% qua thẻ tín dụng<br> | Thông số kỹ thuật: Công nghệ màn hình:\r\nMàn hình Super Retina XDR, Tấm nền OLED, Dynamic Island, Công nghệ ProMotion với tốc độ làm mới thích ứng lên đến 120Hz, Màn hình Luôn Bật, Màn hình HDR, Tỷ lệ tương phản 2.000.000:1 , Màn hình True Tone, Màn hình có dải màu rộng (P3), Haptic Touch<br> Độ phân giải:\r\n1179 x 2556, Chính: 48MP, khẩu độ ƒ/1.78, Ultra Wide: 12MP, khẩu độ ƒ/2.2, Telephoto: 12MP, khẩu độ ƒ/2.8, Camera trước TrueDepth 12MP, khẩu độ ƒ/1.9<br> Kích thước màn hình:\r\n6.1 inch<br> Hệ điều hành:\r\niOS 17<br> Vi xử lý:\r\nA17 Pro<br> Bộ nhớ trong:\r\n128GB<

In [1]:
from langchain_mcp_adapters.client import MultiServerMCPClient
from langchain_mcp_adapters.tools import load_mcp_tools

In [1]:
from langchain_mcp_adapters.client import MultiServerMCPClient
from langchain_mcp_adapters.tools import load_mcp_tools


client = MultiServerMCPClient({
    "hybridsearch": {
        "url": "http://localhost:8000/mcp", 
        "transport": "streamable_http"
    }
})
async with client.session("hybridsearch") as session:
    tools = await load_mcp_tools(session)
    print(tools)
    # call tool
    results = await session.call_tool("hybrid_search",
    {
        "query": "có ip ko",
        "k": 30
    })
    print(results)

[StructuredTool(name='hybrid_search', description="Retrieve restaurant and dish info from the knowledge base.\nSupports:\n- Restaurant details: name, location, cuisine, opening hours\n- Dish details: name, ingredients, dietary info, cooking style\nIncreasing 'k' allows fetching more candidate results for better coverage.", args_schema={'properties': {'query': {'description': 'Restaurant or dish name, location, or type', 'maxLength': 300, 'minLength': 3, 'title': 'Query', 'type': 'string'}, 'k': {'default': 30, 'description': 'Number of top results to retrieve: 30=fast, 50=broad', 'enum': [30, 50], 'title': 'K', 'type': 'integer'}, 'cuisine': {'default': '', 'description': 'Optional cuisine type to filter results', 'maxLength': 50, 'title': 'Cuisine', 'type': 'string'}}, 'required': ['query'], 'type': 'object'}, metadata={'_meta': {'_fastmcp': {'tags': []}}}, response_format='content_and_artifact', coroutine=<function convert_mcp_tool_to_langchain_tool.<locals>.call_tool at 0x0000018DA6

In [8]:
import pandas as pd
file_path="src\data\hoanghamobile_with_summary.csv"
df = pd.read_csv(file_path, nrows=1, encoding="utf-8")
all_columns = df.columns.tolist()
# Define which column will be the content
content_column = "combined_info"
# All other columns go into metadata (exclude content_column)
metadata_columns = [col for col in all_columns if col != content_column]

from langchain_community.document_loaders import CSVLoader
loader = CSVLoader(file_path,
    csv_args={
        'delimiter': ',',
        'quotechar': '"'
    },
    content_columns=content_column,
    metadata_columns=metadata_columns,
    encoding='utf-8'
)
documents = loader.load()